# **Análisis de venta**

Análisis exploratorio de una tabla de ventas proporcionada por negocio.


## **Contexto de los datos**

Cada fila de la tabla representa una venta individual.
El objetivo de este análisis es entender el comportamiento de las ventas
por producto, canal y fecha, y detectar posibles problemas de calidad
en los datos antes de analizarlos.


## **Carga de datos** 

Se carga el archivo CSV proporcionado por negocio para comenzar
la inspección inicial de los datos.


In [16]:
import pandas as pd

df = pd.read_csv("../data/raw/ventas_sucias.csv")

### Inspección inicial

Primera visualización de los datos para entender estructura y contenido general.


In [17]:
df.head()        
"""visualizamos las primeras 5 filas de la tabla. 
confirmamos archivo correcto, posible error al cargar"""

'visualizamos las primeras 5 filas de la tabla. \nconfirmamos archivo correcto, posible error al cargar'

### Estructura y tipos de datos

Se revisa la estructura del DataFrame para identificar tipos de datos,
valores nulos y posibles problemas antes de realizar cualquier limpieza.


In [18]:
df.info()   #informacion del csv, num de filas, columnas, tipos de datos, valores no nulos. 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id_venta  8 non-null      int64  
 1   fecha     8 non-null      object 
 2   cliente   7 non-null      object 
 3   producto  8 non-null      object 
 4   precio    8 non-null      float64
 5   cantidad  7 non-null      float64
 6   canal     7 non-null      object 
dtypes: float64(2), int64(1), object(4)
memory usage: 580.0+ bytes


In [19]:
df.isna().sum() 
"""isna devuelve un dataframe del mismo valor con valores booleanos, 
True -> valores nulos (Nan,None, Nat)
False-> el valor no es Nulo
sum al hacer sum sobre un dataframe True es 1 y False es 0, y pandas suma por columnasº"""

'isna devuelve un dataframe del mismo valor con valores booleanos, \nTrue -> valores nulos (Nan,None, Nat)\nFalse-> el valor no es Nulo\nsum al hacer sum sobre un dataframe True es 1 y False es 0, y pandas suma por columnasº'

## Observaciones iniciales

A partir de la inspección inicial se observan los siguientes problemas:

- Existen valores nulos en las columnas `cliente`, `cantidad` y `canal`.
- La columna `fecha` está almacenada como texto y no como datetime.
- La columna `cantidad` debería contener valores enteros positivos.
- Hay datos que no cumplen las reglas mínimas de negocio para una venta válida.

## <u> Objetivos del Análisis </u>

El objetivo de este análisis es obtener una visión general del rendimiento
de las ventas a partir de la información disponible de la tabla.

En concreto, se debe responder a las siguientes cuestiones:

- ¿Cuántas ventas se han realizado en total?
- ¿Cuáles son los clientes que más y menos han comprado?
- ¿Qué productos se venden con mayor y menor frecuencia?
- ¿Cuánto dinero se ha gastado cada cliente?
- ¿Cuál es el gasto medio por cliente?
- ¿Cómo evolucionan las ventas a lo largo del tiempo?
- ¿Qué día se vendió más y qué día se vendió menos?
- ¿Qué canal de venta genera mayores ingresos?


## Reglas de negocio para el análisis

Para que una venta sea considerada válida en este análisis, debe cumplir las siguientes condiciones:

- Toda venta debe tener un cliente asociado.
- Cantidad vendida debe existir y ser mayor que cero.
- Precio debe ser un valor numérico válido.
- La fecha debe poder interpretarse correctamente como una fecha.
- El canal de venta debe estar informado o clasificarse como desconocido.


## Preparación para la limpieza

Copia del DataFrame original para poder comparar el estado de los datos antes 
y después del proceso de limpieza.


In [20]:
df_raw = df.copy() #creamos copia del dataframe df_raw

## Limpieza de datos

A continuación se realizan las transformaciones necesarias para garantizar
la calidad de los datos antes de realizar el análisis.


### Normalización de la columna fecha

Se convierte la columna `fecha` a tipo datetime para permitir análisis temporal.


In [21]:
df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce") #transforma la columna fecha de object a datetime
df["fecha"].dtype



dtype('<M8[ns]')

Tras la conversión de la columna `fecha` a datetime se detectan un registro con fecha no válida, 
que deberá tratarse según las reglas de negocio definidas.

In [22]:
df["fecha"].isna().sum()  # Cuantas fechas invalidas hay??


np.int64(1)

Se detecta un registro con fecha no válida que deberá eliminarse.

In [23]:
df = df.dropna(subset=["fecha"]).copy()  #Elimina las filas que tengan valores nulos en la columna fecha y crea una nueva copia
"""dropna NO modifica df por defecto, devuelve un nuevo DataFrame, por eso reasignamos a df con df = """

'dropna NO modifica df por defecto, devuelve un nuevo DataFrame, por eso reasignamos a df con df = '

In [24]:
df["fecha"].isna().sum()   
df.shape                 #comprobacion



(7, 7)

Se eliminan las ventas con fecha inválida, ya que no pueden utilizarse
de forma fiable en el análisis.


### Validación y limpieza de `cliente`

No se aceptan ventas sin cliente informado, por lo que se eliminarán estos registros.


In [25]:
df["cliente"].isna().sum() #cuántos clientes faltan??

np.int64(1)

In [26]:
df[df["cliente"].isna()][["id_venta", "fecha", "cliente", "producto", "precio", "cantidad", "canal"]]  #Que clientes son los que faltan??

,id_venta,fecha,cliente,producto,precio,cantidad,canal
2,3,2024-01-06,NaN,Monitor,180.0,1.0,web


In [27]:
df = df.dropna(subset=["cliente"]).copy() #Eliminamos la fila con cliente es null.
df["cliente"].isna().sum(), df.shape #Verfificamos.


(np.int64(0), (6, 7))

### Validación y limpieza de `cantidad`

No se aceptan ventas sin cantidad o con cantidad menor o igual a 0.


In [28]:
df["cantidad"].isna().sum() #nulos en cantidad


np.int64(1)

In [29]:
df[df["cantidad"].isna()][["id_venta", "fecha", "cliente", "producto", "precio", "cantidad", "canal"]] #ver filas con cantidad nula


,id_venta,fecha,cliente,producto,precio,cantidad,canal
3,4,2024-01-06,Marta Pérez,Teclado,25.5,NaN,tienda


In [30]:
df[df["cantidad"] <= 0][["id_venta", "fecha", "cliente", "producto", "precio", "cantidad", "canal"]] 
#ver filas con cantidad <= 0


,id_venta,fecha,cliente,producto,precio,cantidad,canal
5,6,2024-01-07,Ana López,Ratón,15.0,-1.0,web


In [ ]:
df = df.dropna(subset=["cantidad"]).copy()
df = df[df["cantidad"] > 0].copy()
(df["cantidad"].isna().sum(), (df["cantidad"] <= 0).sum(), df.shape) #limpieza y verificación.


(np.int64(0), np.int64(0), (4, 7))

### Métrica básica: importe por venta

Se crea la métrica `importe = precio * cantidad` para analizar ventas.


In [ ]:
df["importe"] = df["precio"] * df["cantidad"]   #creamos la columna importe, suma de importes totales.
df[["id_venta", "fecha", "cliente", "producto", "precio", "cantidad", "importe", "canal"]] #visualización de todas las columnas.


,id_venta,fecha,cliente,producto,precio,cantidad,importe,canal
0,1,2024-01-05,Ana López,Teclado,25.5,2.0,51.0,web
1,2,2024-01-05,Carlos Ruiz,Ratón,15.0,1.0,15.0,tienda
6,7,2024-01-08,Carlos Ruiz,Monitor,180.0,1.0,180.0,NaN
7,8,2024-01-08,Ana López,Teclado,25.0,2.0,50.0,web


## Análisis de negocio

Con los datos ya limpios y la métrica `importe = precio × cantidad` calculada, se responden las preguntas de negocio solicitadas.


### 1. Ventas totales y facturación

- **Ventas totales**: número de registros (filas).
- **Facturación total**: suma del importe de todas las ventas.


In [34]:
ventas_totales = len(df)
facturacion_total = df["importe"].sum()

ventas_totales, facturacion_total


(4, np.float64(296.0))

### 2. Clientes con mayor y menor gasto

Se agrupa por cliente y se suma el importe total para identificar el top y el bottom.


In [35]:
gasto_por_cliente = (
    df.groupby("cliente")["importe"]
    .sum()
    .sort_values(ascending=False)
)

gasto_por_cliente


cliente
Carlos Ruiz    195.0
Ana López      101.0
Name: importe, dtype: float64

In [36]:
cliente_mas_gasta = gasto_por_cliente.index[0], gasto_por_cliente.iloc[0]
cliente_menos_gasta = gasto_por_cliente.index[-1], gasto_por_cliente.iloc[-1]

cliente_mas_gasta, cliente_menos_gasta


(('Carlos Ruiz', np.float64(195.0)), ('Ana López', np.float64(101.0)))

### 3. Productos más y menos vendidos (frecuencia)

Se cuenta cuántas ventas hay por producto.


In [37]:
ventas_por_producto = (
    df.groupby("producto")
    .size()
    .sort_values(ascending=False)
)

ventas_por_producto


producto
Teclado    2
Monitor    1
Ratón      1
dtype: int64

In [ ]:
producto_mas_vendido = ventas_por_producto.index[0], ventas_por_producto.iloc[0]
producto_menos_vendido = ventas_por_producto.index[-1], ventas_por_producto.iloc[-1]

producto_mas_vendido, producto_menos_vendido
